In [1]:
import sys
import logging, os
from time import time
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib notebook

import ipyparallel as ipp

from bem import Electrodes, Sphere, Cylinder, Mesh, Grid, Configuration, Result
from bem.formats import stl

# Import mesh from stl

In [2]:
# base file name for outputs and inputs is the script name
prefix = "SimpleTrap"

In [3]:
a0 = 40e-6    # Distance from ion to electrode is 40 µm.
unit = 1e-6   # unit of the stl file is µm.

# load electrode faces from colored stl
# s_nta is intermediate processed stl file.
s_nta = stl.read_stl(open("%s.stl" % prefix, "rb"))
print("Import stl:",os.path.abspath("./"+prefix+".stl"),"\n")
print("Electrode colors (numbers):\n")
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=a0/unit,rename={1:"1"}))   
# after scale, unit is a0

Import stl: /Users/nem0x/Documents/Pyckages/bem-savio/examples/SimpleTrap/SimpleTrap.stl 

Electrode colors (numbers):

dropping 0
dropping 9495
dropping 17962
dropping 18994
dropping 18869
dropping 20943
dropping 18129
mesh.gather() not working properly, be sure to have valid input


## The formal rename of electrode. 
Assign each electrode a string name instead of its color coding. Use the numbers you get above.  
`stl.stl_to_mesh()` prints normal vectors (different faces) in each electrode.

In [4]:
print(len(s_nta), type(s_nta),"\n")
# s_nta is a length 3 tuple. (normal, triangle, attribute) 
# Normal direction of each triangle, three vetice of triangles, coding number of colors.
print("Triangles:",len(s_nta[0]),"\nColors:",len(s_nta[2]),"\n")    # This isn't right.

# stl_to_mesh() only assigns names and does scaling, doing no triangulation to stl mesh.
# "scale=a0/unit" only scales dimensionless
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=a0/unit,
    rename={9495: "DC1", 
            17962: "DC3", 
            18994: "DC5",
            18869: "DC2", 
            20943: "RF", 
            18129: "DC4"
           }, quiet=False))    


3 <class 'tuple'> 

Triangles: 440 
Colors: 440 

dropping 0
1 planes in electrode DC1
normals vectors:
 [[ 0. -0.  1.]]
1 planes in electrode DC3
normals vectors:
 [[0. 0. 1.]]
1 planes in electrode DC5
normals vectors:
 [[0. 0. 1.]]
1 planes in electrode DC2
normals vectors:
 [[0. 0. 1.]]
1 planes in electrode RF
normals vectors:
 [[0. 0. 1.]]
1 planes in electrode DC4
normals vectors:
 [[ 0. -0.  1.]]


# Generate triangle mesh with constraints

The meshes are 2-dimensional triangles on the surface of electrodes. The region enclosed by constraint shape can have finer mesh. Triangulation is done by `triangle` C library.

In [5]:
# set .1 max area within 3
# areas_from_constraints specifies sphere with finer mesh inside it.
mesh.areas_from_constraints(Sphere(center=np.array([0, 0, 1.]),
           radius=2, inside=0.2, outside=10))    # "inside", "outside" set different mesh densities.
# retriangulate quality and quiet with areas
mesh.triangulate(opts="q20Q", new=False)
# save base mesh to vtk
mesh.to_vtk(prefix)
print("Output vtk:",os.path.abspath("./"+prefix+".vtk"))    # output path

start triangulate DC1
('final opts', 'q20Qzra')
finish triangulate DC1
start triangulate DC3
('final opts', 'q20Qzra')
finish triangulate DC3
start triangulate DC5
('final opts', 'q20Qzra')
finish triangulate DC5
start triangulate DC2
('final opts', 'q20Qzra')
finish triangulate DC2
start triangulate RF
('final opts', 'q20Qzra')
finish triangulate RF
start triangulate DC4
('final opts', 'q20Qzra')
finish triangulate DC4
Output vtk: /Users/nem0x/Documents/Pyckages/bem-savio/examples/SimpleTrap/SimpleTrap.vtk


## Plot mesh

Plot mesh using pyvista plotter in a separate window.
On macOS perhaps Linux as well, the window doesn't close properly after clicking close button, but the code can continue to run.

In [6]:
# Plot triangle meshes.
mesh.plot()

# Main boundary element calculations

In `run_job` function, `job` is `Configuration` instance and `grid` is discretirized spatial grid (not the mesh). The general workflow (also the routine of BEM method) are:  
1. `solve_singularities()` solves charge distributions by iterative methods to make it consistent with one electrode at 1V and others at 0V (unit potentials). `adapt_mesh()` refines meshes adaptively to achieve certain precision while solving sigulartities.
2. Compute potentials on given grid points by `simulate()`, based on the charge distributions gotten previously.
3. Potential data of each unit potential are saved seperately to a `Result` instance, and also export to VTK files.
4. Return total accumulated charge per electrode in the end.

Major calculations calls `fastlap` C library which uses a pre-conditioned, adaptive, multipole-accelerated algorithm for solving Laplace problem. Two parameters control multipole acceleration.
+ num_mom, the number of multipole
+ num_lev, the number of levels in the hierarchical spatial decomposition.  
num_lev=1 means direct computation without multipole acceleration. See fastlap ug.pdf and README.rst.

In [6]:
# Define calculation function.
def run_job(args):
    # job is Configuration instance.
    job, grid, prefix = args
    # refine twice adaptively with increasing number of triangles, min angle 25 deg.
    job.adapt_mesh(triangles=4e2, opts="q25Q")
    job.adapt_mesh(triangles=1e3, opts="q25Q")
    # solve for surface charges
    job.solve_singularities(num_mom=4, num_lev=3)
    # get potentials and fields
    result = job.simulate(grid, field=job.name=="RF", num_lev=2)    # For "RF", field=True computes the field.
    result.to_vtk(prefix)
    print("finished job %s" % job.name)
    return job.collect_charges()

## Define grid

Create a grid in unit of scaled length `l`. Only choose the interested region (trap center) to save time.

For reference, to compute Seidelin trap, grid shape = (60, 60, 60) takes 266 s, while shape = (150, 150, 150) takes 3369 s.

In [8]:
# grid to evalute potential and fields atCreate a grid in unit of scaled length l. Only choose the interested region (trap center) to save time.
s = 0.08
Lx, Ly, Lz = 2, 2, 2    # in the unit of scaled length l
sx, sy, sz = s, s, s
# ni is grid point number, si is step size. Thus to fix size on i direction you need to fix ni*si.
nx, ny, nz = [2*np.ceil(L/2.0/ss).astype('int')+1 for ss, L in zip((sx, sy, sz), (Lx, Ly, Lz))]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(0, 0, 1.5), step=(sx, sy, sz), shape=(nx, ny, nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("Grid origin/l:", grid.get_origin()[0])

Size/l: 2 2 2
Step/l: 0.08 0.08 0.08
Shape (grid point numbers): 27 27 27
Grid origin/l: -1.04


# Parallel computation using ipyparallel, which is compatible with Savio BRC-HPC

In [8]:
# create job list
jobs = list(Configuration.select(mesh,"DC.*","RF"))

# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


INFO:ipyparallel.cluster.cluster.1666855509-orme:Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

run_job:   0%|          | 0/6 [00:00<?, ?tasks/s]

Computing time: 11.055103 s


# View result in 3D

In [15]:
# base file name for outputs and inputs is the script name
prefix = "SimpleTrap"

view exported base mesh vtk file

In [7]:
Result.view(prefix, '') # don't add anything between '' for mesh view

view simulation result of an electrode

In [ ]:
Result.view(prefix, 'RF') # add electrode name between '' for result view

# Load result

In [3]:
rf_result = Result.from_vtk(prefix, 'RF')